<a href="https://colab.research.google.com/github/herasj/Constructor_Imagenes/blob/master/Proyecto1_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Librerias
#Iniciar de primero
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import figure
import numpy as np
from PIL import Image,ImageDraw
from string import Template
import copy
import math
import heapq
import random

# Procesamiento de imagenes



### Recorte de Imagenes

Esta función recorta la imagen principal y guarda cada parte con un numero aleatorio que va desde el 0 hasta el 8.

La imagen original debe subirse en el root del directorio y debe llamarse **img.bmp**.

Para mayor orden se debe crear una carpeta llamada **crop** en donde se guardaran las imagenes recortadas.

**NOTA: Si no se crea dicha carpeta el algoritmo de recorte fallará**

**NOTA2: Si desea recortar la imagen con otro programa puede omitir esta pieza de código**


In [0]:
picknum=[0,1,2,3,4,5,6,7,8]
size=[] #Inicializar lista

#Abrir imagen principal
im = Image.open('./img.bmp')# Imagen Cuadrada !
dim = (im.height) #Las dimensiones de la imagen

#Coordenadas del recorte
size.append(0)
size.append(math.floor(dim/3))
size.append(math.floor(2*dim/3))
size.append(dim)
cont=0;
for y in range(0,3):
    for x in range(0,3):
        num=random.choice(picknum)
        t = Template('./crop/$name.bmp') #Path de guardado
        path = t.substitute(name=num) #Template string
        up=size[y]#0,166,333
        l=size[x]#166,333,500
        r=l+size[1] #x+w
        dw=up+size[1] #y+h
        recorte = im.crop((l,up,r,dw)) #(x,y,x+w,y+h) coordenadas
        recorte.save(path) #Guardar imagen recortada
        picknum.remove(num)
        # cont=cont+1

#Generar una imagen negra para el cuadro vacio
black_path = './black.jpg'
im = Image.new('L', (100,100), 0)
im.save(black_path)

## Operaciones

### Get Borders
Para mayor facilidad la imagen se vuelve monocromatica, en la siguiente funcion se obtiene el valor absoluto de cada pixel en los bordes *El cual puede ir desde 0 a 255* dicha información se guarda en una lista para ser procesada más tarde.

In [0]:
def getborders(img,h):
    bordes=[] #Lista que contiene las demás
    up=[]
    left=[]
    right=[]
    down=[]

    #Bordes
    for x in range (0, h):
        up.append(abs(img[x,0])) #[X,Y]  Superior -img[x,1]
        down.append(abs(img[x,h-1])) #Inferior -img[x,h-2]
        left.append(abs(img[0,x])) #Izquierda -img[1,x]
        right.append(abs(img[h-1,x])) #Derecha -img[h-2,x]
        #Obtiene la diferencia entre los pixeles de cada borde

    #Estructura de bordes [[up],[down],[left],[right]]
    bordes.append(up);bordes.append(down);bordes.append(left);bordes.append(right)
    return bordes


### Cargar imagenes
La siguiente función carga todas las imagenes recortadas.

In [0]:
#Funcion de carga
def cargarImgs(color):
    imgs=[]
    for x in range(0,9):
        t = Template('./crop/$name.bmp') #Path de carga
        path = t.substitute(name=x) #Template string
        if(color==False):
            imgs.append(Image.open(path).convert("L"))
        else:
             imgs.append(Image.open(path))
    return imgs


def mostrar(img1,img2,h):
    b1=[]
    b2=[]

    for x in range (0, h):
        b1.append(img1[h-1,x]) #[X,Y]

    for y in range (0, h):
        b2.append(img2[0,y]) #[X,Y]
    resta=[]
    for z in range(0,len(b1)):
        resta.append(abs(b1[z]-b2[z]))
    
    print("Borde 1")
    print(b1)
    print("Borde 2")
    print(b2)
    print("Resta de bordes")
    print(resta)

### Operaciones de lista
Un conjunto de operaciones que se realizan sobre la lista de bordes para poder formar la imagen original.

In [0]:
def sumar_subl(l,h): #Sumar los valores de cada sublista
    newl=[]
    temp=0
    for x in range(0,4):
        for y in range(0,h):
            temp=temp+l[x][y]
        newl.append(copy.deepcopy(temp))
        temp=0
    return newl

def restar_l(l1,l2,h): #Restar dos listas de bordes
    lista_resta=[] #Lista que contendrá todas las diferencias
    r_l=[] #Almacena la diferencia entre el derecho de l1 y el izquierdo de l2
    u_d=[] #Almacena la diferencia entre el superior de l1 y el inferior de l2
    d_u=[] #Almacena la diferencia entre el inferior de l1 y el superior de l2
    l_r=[] #Almacena la diferencia entre el izquierdo de l1 y el derecho de l2

    for x in range(0,h):
        r_l.append(abs(l1[3][x]-l2[2][x]))
        u_d.append(abs(l1[0][x]-l2[1][x]))
        d_u.append(abs(l1[1][x]-l2[0][x]))
        l_r.append(abs(l1[2][x]-l2[3][x]))

    #Añadir los datos a la lista principal
    lista_resta.append(copy.deepcopy(r_l));
    lista_resta.append(copy.deepcopy(u_d));
    lista_resta.append(copy.deepcopy(d_u));
    lista_resta.append(copy.deepcopy(l_r));

    r_l.clear();u_d.clear();d_u.clear();l_r.clear(); #Eliminar datos innecesarios

    return lista_resta

def menor(list):
    temp=[99999,-1] #[Menor, posición]
    for x in range(0,len(list)):
        if(list[x]<temp[0]):
            temp[0]=list[x]
            temp[1]=x
    return temp

def mueve(grid,n1,n2,mov): 
  #Si el mov es 0, n2 queda a la derecha de n1
  #Si el mov es 1, n2 queda arriba de n1
  #Si el mov es 2, n2 queda debajo de n1
  #Si el mov es 3, n2 queda a la izquierda de n1
  esta1=esta(grid,n1)
  esta2=esta(grid,n2)

  if (mov==0): # n1->n2
    if((esta1[0]!=-1) and (esta2[0]==-1)): #Si está n1 pero n2 no
      if(grid[esta1[0]][2]==-9):
        grid[esta1[0]][2]=n2 #Se inserta n2 al lado de n1
    elif ((esta1[0]==-1) and (esta2[0]!=-1)):#Si està n2 pero no n1
      if(grid[esta1[0]][0]==-9): 
        grid[esta1[0]][0]=n1 #Se inserta n1 a la izq de n2
    # elif ((esta1[0]==-1) and (esta2[0]==-1)):#No esta n1 y n2
        # #Se intenta añadir n1 y n2 al grid vacio NUNCA SUCEDE
        # if(len(grid[0])<=1):
        #     grid[0].append(n1);grid[0].append(n2) 
        # elif(len(grid[1])<=1):
        #     grid[1].append(n1);grid[1].append(n2)
        # elif(len(grid[2])<=1):
        #     grid[2].append(n1);grid[2].append(n2)
        # else:
        #     print("ERROR: No hay espacio en el grid para n1= ",n1," y n2= ",n2, "con mov= ",mov)
        #     print(grid)
  elif mov==1: #n2 arriba de n1
    if((esta1[0]!=-1) and (esta2[0]==-1)): #Si está n1 pero n2 no
      if(grid[0][esta1[1]]==-9):
        grid[0][esta1[1]]==n2 #Se inserta n2 arriba de n1
    elif ((esta1[0]==-1) and (esta2[0]!=-1)):#Si està n2 pero no n1
      if(grid[2][esta1[1]]==-9): 
        grid[2][esta1[1]]=n1 #Se inserta n1 debajo de n2
    # elif ((esta1[0]==-1) and (esta2[0]==-1)):#No esta n1 y n2
        #Se intenta añadir n1 y n2 al grid vacio
        # if((len(grid[0])<=2)and (len(grid[1])<=2)):
        #     grid[0].append(n2);grid[1].append(n1) 
        # elif((len(grid[1])<=2)and (len(grid[2])<=2)):
        #     grid[1].append(n2);grid[2].append(n1)
        # else:
        #     print("ERROR: No hay espacio en el grid para n1= ",n1," y n2= ",n2, "con mov= ",mov)
        #     print(grid)
  elif mov==2:#n2 debajo de n1
    if((esta1[0]!=-1) and (esta2[0]==-1)): #Si está n1 pero n2 no
        if(grid[2][esta1[1]]==-9):
          grid[2][esta1[1]]=n2 #Se inserta n2 debajo de n1
    elif ((esta1[0]==-1) and (esta2[0]!=-1)):#Si està n2 pero no n1
        if(grid[0][esta1[1]]==-9):
          grid[0][esta1[1]]=n1 #Se inserta n1 arriba de n2
    # elif ((esta1[0]==-1) and (esta2[0]==-1)):#No esta n1 y n2
        #Se intenta añadir n1 y n2 al grid vacio
        # if((len(grid[0])<=2)and (len(grid[1])<=2)):
        #     grid[0].append(n1);grid[1].append(n2) 
        # elif((len(grid[1])<=2)and (len(grid[2])<=2)):
        #     grid[1].append(n1);grid[2].append(n2)
        # else:
        #     print("ERROR: No hay espacio en el grid para n1= ",n1," y n2= ",n2, "con mov= ",mov)
        #     print(grid)
  else: #n2 izq de n1
      if((esta1[0]!=-1) and (esta2[0]==-1)): #Si está n1 pero n2 no
        if(grid[esta1[0]][0]==-9):
          grid[esta1[0]][0]=n2 #Se inserta n2 a la izq de n1
      elif ((esta1[0]==-1) and (esta2[0]!=-1)):#Si està n2 pero no n1
        if(grid[esta1[0]][2]==-9):
          grid[esta1[0]][2]=n1 #Se inserta n1 a la der de n2
      # elif ((esta1[0]==-1) and (esta2[0]==-1)):#No esta n1 y n2
          # #Se intenta añadir n1 y n2 al grid vacio
          # if(len(grid[0])<=1):
          #     grid[0].append(n1);grid[0].append(n2) 
          # elif(len(grid[1])<=1):
          #     grid[1].append(n1);grid[1].append(n2)
          # elif(len(grid[2])<=1):
          #     grid[2].append(n1);grid[2].append(n2)
          # else:
          #     print("ERROR: No hay espacio en el grid para n1= ",n1," y n2= ",n2, "con mov= ",mov)
          #     print(grid)
          
  return grid

def esta (grid,n):
    res=[-1,-1]
    if((len(grid[0])==0) and (len(grid[1])==0) and (len(grid[2])==0)): 
        return res

    for x in range(0,len(grid)):
        if(len(grid[x])!=0):
            for y in range(0,len(grid[x])):
                if(grid[x][y]==n):
                    res[0]=x
                    res[1]=y

    return res
def centrar (grid,centro):
    #Si el mov es 0, n2 queda a la derecha de n1
    #Si el mov es 1, n2 queda arriba de n1
    #Si el mov es 2, n2 queda debajo de n1
    #Si el mov es 3, n2 queda a la izquierda de n1

    grid[1].append(centro[0])
    for x in range(1,5):
        if(centro[x][0]==0):
            grid[1].append(centro[x][1])
        elif (centro[x][0]==1):
            grid[0].append(-9)
            grid[0].insert(1,centro[x][1])
            grid[0].append(-9)
        elif (centro[x][0]==2):
            grid[2].append(-9)
            grid[2].insert(1,centro[x][1])
            grid[2].append(-9)
        elif (centro[x][0]==3):
            grid[1].insert(0,centro[x][1])
    print("Asi quedó la grid")
    print(grid)
    return grid

def mayor_lista(lista):
  may=0
  pos=-1
  for x in range(0,4):
    if(lista[x][2]>may):
      may=lista[x][2]
      pos=x
  return [may,pos]

def sum_lista(lista):
  sumval=0
  for x in lista:
    sumval=sumval+x[2]
  return sumval

def duplicados(lista_init):
  # print(' ')
  listaindex=[]
  for x in lista_init:
    listaindex.append(x[0])
  seen = set()
  # print('Lista: ',listaindex)
  for x in listaindex:
    if x in seen: return True
    seen.add(x)
  return False

## Ordenar imagenes
Archivo principal el cual se encarga de procesar la imagen apoyandose en las funciones anteriormente descritas

**NOTA: Se debe crear una carpeta llamada orden, allí se guardaran las imagenes ordenadas, si no crea dicha carpeta el argoritmo fallará**

In [0]:
gridlist=[[],[],[]]
#NOTA: ARREGLAR ESQUINAS
centro=[]
ady=[] #[label,up,down,left,right]
targets=[0,1,2,3,4,5,6,7,8] #Lista de objetivos

imgs=cargarImgs(False) #Carga todas las imagenes en /crop
imgscolor=cargarImgs(True) #Carga las imagenes con colores
org=Image.open('./img.bmp')
dim=imgs[0].height; #Altura de las imagenes
umbral=621 #Umbral de elección
#Buscar el centro
centro_temp=[]
centro_prob=[[-1,-1,99999],[-1,-1,99999],[-1,-1,99999],[-1,-1,99999]]
pos_temp=0
centro=[]
for x in range (0,10):
   #Posible centro
  # print('Posible centro: en x= ',x-1,' ',centro_temp)
  if (len(centro_temp)==4):
    dup=duplicados(centro_temp)
    if(dup==True):
      # print('Descartado por duplicación: ',x-1)
      # print(centro_temp)
      centro_temp.clear()
    else:
      sum_temp=sum_lista(centro_temp)
      sum_target=sum_lista(centro_prob)
      if (sum_temp<sum_target):
        # print('En x: ',x, '  ',centro_temp)
        # print('sumtemp: ',sum_temp,' sumtarget: ',sum_target)
        # print(' ')
        centro_prob.clear()
        centro_prob=copy.deepcopy(centro_temp)
        centro_temp.clear()
        pos_temp=x-1
      else:
        centro_temp.clear()
  else:
    centro_temp.clear()
  if (x<=8):
      # if(len(centro)!=5):
      #     centro.clear()
      # else:
      #     print("Se rompió el for")
      #     break
    # centro.append(copy.deepcopy(x)) #Añadir el nodo a comparar
    target=imgs[x].load()
    target_b=getborders(target,dim)
    ady.clear()
    for y in range(0,9):
      if (y!=x):
          test=imgs[y].load()
          test_b=getborders(test,dim)
          resta = restar_l(target_b,test_b,dim)
          suma=sumar_subl(resta,dim)
          nro_menor=menor(suma)
          # print('Suma de cada diferencia target = ',str(x),' y= ',str(y),' ',suma)
          # print('numeros menores: ',nro_menor)
          if (nro_menor[0]>0):
            # print('Entra')
            if(len(centro_temp)<4):
              centro_temp.append([nro_menor[1],y,nro_menor[0]])
            else:
              [mayor,pos]=mayor_lista(centro_temp)
              if(nro_menor[0]<mayor):
                centro_temp[pos]=[nro_menor[1],y,nro_menor[0]]
              # print('ADD CENTRO: ',nro_menor,' y: ',y )
              # centro.append([nro_menor[1],y])

centro.clear()
# print('Posición: ',pos_temp)
# print('Centro Prob: ',centro_prob)
centro.append(pos_temp)
bordes=[]
for lista in centro_prob:
  centro.append([lista[0],lista[1]])
# print('El centro es : ',centro)
[umbral,w] = mayor_lista(centro_prob)
umbral=int(umbral)+1
print('umbral: ', umbral)
                     
if(len(centro)<5): # [Nodo, [Posibles centros]]
    print('ERROR: No se reconoce centro')
else:
    gridlist=centrar(gridlist,centro)

print('grid centrada :) ',gridlist)
lista_recorrida=[]
lista_recorrida.append(pos_temp)
for x in centro_prob:
  lista_recorrida.append(x[1])

bordes.append(gridlist[0][1])
bordes.append(gridlist[2][1])
# print('Bordes: ',bordes)
#Acomodar casillas restantes
# for x in range (0,9):
#   if x not in lista_recorrida :
#     target=imgs[x].load()
#     target_b=getborders(target,dim)
#     ady.clear()
#     for y in range(0,9):
#         if (y!=x):
#             test=imgs[y].load()
#             test_b=getborders(test,dim)
#             resta = restar_l(target_b,test_b,dim)
#             suma=sumar_subl(resta,dim)
#             nro_menor=menor(suma)
#             # print('Suma de cada diferencia target = ',str(x),' y= ',str(y),' ',suma)
#             if (nro_menor[0]<=umbral and nro_menor[0]>0):
#               # print('Parametros: ',x,' ',y,' ',nro_menor)
#               gridlist=mueve(gridlist,x,y,nro_menor[1])
#               # print('Progreso de la grid: ',gridlist)

for x in range(0,2):
  target=imgs[bordes[x]].load()
  target_b=getborders(target,dim)
  for borde in range(0,2):
    b_totales=[999999,-2]
    for y in range(0,9):
      if y not in lista_recorrida:
        test=imgs[y].load()
        test_b=getborders(test,dim)
        resta = restar_l(target_b,test_b,dim)
        suma=sumar_subl(resta,dim)

        # print('x: ',bordes[x],' y: ',y, ' borde: ',borde)
        # print('suma: ',suma)
        # print('')

        if borde==0:
          if(suma[3]<b_totales[0]):
            b_totales[0]=suma[3]
            b_totales[1]=y
        else:
          if(suma[0]<b_totales[0]):
            b_totales[0]=suma[0]
            b_totales[1]=y
    
    lista_recorrida.append(b_totales[1])
    if(borde==0 and x==0):
      gridlist[0][0]=b_totales[1]
    elif(borde==1 and x==0):
      gridlist[0][2]=b_totales[1]
    elif(borde==0 and x==1):
      gridlist[2][0]=b_totales[1]
    elif(borde==1 and x==1):
      gridlist[2][2]=b_totales[1]



#Imprimir resultado
# tiles=[]
# tiles.extend(gridlist[0])
# tiles.extend(gridlist[1])
# tiles.extend(gridlist[2])
# for x in targets:
#   if x not in tiles:
#     for z in range(0,3):
#       for w in range(0,3):
#         if(gridlist[z][w]==-9):
#           gridlist[z][w]=x
#           break
print("grid final")
print(gridlist)

contador=0
for x in range(0,len(gridlist)):
    for y in range(0,len(gridlist)):
        t = Template('./orden/$name.bmp') #Path de guardado
        path = t.substitute(name=contador) #Template string
        imgscolor[gridlist[x][y]].save(path) #Guardar ordenados
        contador=contador+1
print('Imagen completa')
f = open("grid.txt", "w")
for x in gridlist:
  for y in x: 
    f.write(str(y)+',')
  f.write(';') 
f.close()


umbral:  2402
Asi quedó la grid
[[-9, 0, -9], [6, 5, 8], [-9, 1, -9]]
grid centrada :)  [[-9, 0, -9], [6, 5, 8], [-9, 1, -9]]
grid final
[[3, 0, 2], [6, 5, 8], [4, 1, 7]]
Imagen completa


# Main

In [0]:
plt_img=[] #Lista con las imagenes ordenadas
plt_img.append(mpimg.imread('./black.jpg'))

# f = open("grid.txt", "r")
# archivo = f.read()
# f.close()
# gridfile = archivo.split(';')
# valores= []
# for x in gridfile:
#   valores.append(x.split(','))
# gridfinal=[]
# for y in range(0,3):
#     gridfinal.append([int(valores[0][y]),int(valores[1][y]),int(valores[2][y])])
  
# 
# grid=gridfinal

# Para usar la grid de la imagenes 
for x in range(0,9):
      t = Template('./orden/$name.bmp') #Path de carga
      path = t.substitute(name=x) #Template string
      plt_img.append(mpimg.imread(path));

# Busqueda A*
grids = []
Resolver=False

while(Resolver == False):
  grid =np.array([[0, 0, 0],[0, 0, 0],[0, 0, 0],])
  picknum=[0,1,2,3,4,5,6,7,8]
  for x in range(0,3):
    for y in range(0,3):
      num=random.choice(picknum) #Random choice
      grid[x][y]=num
      picknum.remove(num)
  Resolver = solvable(grid)

print("Grid inicial: ")
print(grid)
goal =np.array([[1, 2, 3],[4, 5, 6],[7, 8, 0],])
valF = heurDist(grid)
createGrid(grid, 0, valF, grids, None)
final = a_star_search(grids[0], goal,grids)
path = []
path.append(final.tiles)
while(final.parent != None):
  path.append(final.parent.tiles)
  final = final.parent
path.reverse()
print('Numero de movimientos : ' + str(len(path) - 1))
print('')

# for i in path: Mostrar datos
#   print(i)
#   print('')

# Plot
print("Recorrido")
print('')
for z in range (0,len(path)): 
  fig = plt.figure(figsize=(4,4))
  axarr = [fig.add_subplot(3,3,i+1) for i in range(9)]
  cont=0;
  target = path[z]
  for x in range(0,3):
    for y in range(0,3):
      #Plot Image
      axarr[cont].imshow(plt_img[target[x,y]])
      axarr[cont].axis('off')
      axarr[cont].set_xticklabels([])
      axarr[cont].set_yticklabels([])
      cont=cont+1
      fig.subplots_adjust(wspace=0.01, hspace=0.01)
cont=1;

for x in range(0,9):
  axarr[x].imshow(plt_img[cont])
  axarr[x].axis('off')
  axarr[x].set_xticklabels([])
  axarr[x].set_yticklabels([])
  cont=cont+1
  fig.subplots_adjust(wspace=0.01, hspace=0.01)      


valores:  [[3, 6, 4], [0, 5, 1], [2, 8, 7]]
Grid inicial: 
[[6 4 8]
 [1 7 2]
 [0 3 5]]


KeyboardInterrupt: ignored

# Logica del Puzzle

In [0]:
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, priority, item):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)

#Grid class with all tiles positions and exact distance from
#the start (gVal) and estimate distance to the goal plus gVal (fVal)
class gridObj:
  def __init__(self,tiles,index,gVal,fVal,parent):
    self.tiles = tiles
    self.index = index
    self.gVal = gVal
    self.fVal = fVal
    self.parent = parent
  
  def __lt__(self,other):
    valueF = self.gVal + self.fVal
    valueO = other.gVal + other.fVal
    if(valueF>valueO):
      return 0
    else:
      return 1
  
  def setgVal(self,gVal):
    self.gVal = gVal
  
  def setParent(self,parent):
    self.parent = parent

#Creates a grid if it doesn't exist already and add it to the list of grids, else return the existing grid 
def createGrid(current,gVal,fVal,grids,parent):
  if len(grids)>0:
    for i in grids:
      if ((i.tiles==current).all()):
        return i
  tempGrid = gridObj(current, len(grids), gVal, fVal, parent)
  grids.append(tempGrid)
  return tempGrid

#This function takes the value of a grid position and 
# return an npArray with the proper position in the goal Grid 
def valToPos(val):
    pos = np.zeros((2))
    if (val==0):
      x = 2
      y = 2
    else:  
      if val <= 3 and val!=0 :
        x = 0
        y = val -1 
      elif val <= 6 :
        x = 1
        y = val -4
      elif val<=8:
        x = 2
        y = val - 7
      
    pos[0] = int (x)
    pos[1] = int (y)
    return pos

def returnEmpty(grid):
  empty = np.zeros((2))
  for i in range(0,3):
    for j in range(0,3):
      if (grid[i,j]==0):
        empty[0] = i
        empty[1] = j
        return empty #Esa linea equivale a un break
  return empty

#return a list of the neighbors for a grid, creates the necessary grids also
def neighbors (grid,gVal,grids,parent):
  empty = returnEmpty(grid) #Get the position of the empty spot 
  x = empty[0]
  y = empty[1]
  x = int(x)
  y = int(y)
  neighbors = []
  
  if (x+1<3):
    tempGrid = np.copy(grid) #copy the current grid
    tempPos = grid[x+1,y] #Save one of the positions to change
    tempGrid[x+1,y] = tempGrid[x,y] #change the value of the position saved before
    tempGrid[x,y] = tempPos #Change the value of the other position
    neighbors.append(createGrid(tempGrid, gVal+1, heurDist(tempGrid), grids, parent))
  if (x-1>=0):
    tempGrid = np.copy(grid) #copy the current grid
    tempPos = grid[x-1,y] #Save one of the positions to change
    tempGrid[x-1,y] = tempGrid[x,y] #change the value of the position saved before
    tempGrid[x,y] = tempPos #Change the value of the other position
    neighbors.append(createGrid(tempGrid,gVal+1,heurDist(tempGrid), grids, parent))
  if (y+1<3):
    tempGrid = np.copy(grid) #copy the current grid
    tempPos = grid[x,y+1] #Save one of the positions to change
    tempGrid[x,y+1] = tempGrid[x,y] #change the value of the position saved before
    tempGrid[x,y] = tempPos #Change the value of the other position
    neighbors.append(createGrid(tempGrid,gVal+1,heurDist(tempGrid), grids, parent))
  if (y-1>=0):
    tempGrid = np.copy(grid) #copy the current grid
    tempPos = grid[x,y-1] #Save one of the positions to change
    tempGrid[x,y-1] = tempGrid[x,y] #change the value of the position saved before
    tempGrid[x,y] = tempPos #Change the value of the other position
    neighbors.append(createGrid(tempGrid,gVal+1,heurDist(tempGrid), grids, parent))
  return neighbors

#iterates through the grid and accumulate the distance between
#every tile in the current grid to the correct tile position in the final grid
def heurDist(grid):
  dist = 0
  if(grid[0,0]!=1):
    dist = 5
  val = 1
  for i in range(0,3):
    for j in range(0,3):
      pos = valToPos(grid[i,j])
      dist = dist + (abs(pos[0]-i) + abs(pos[1]-j))
      val = val + 1
  #print(dist)
  dist = dist + linearConflict(grid) - 1
  if(dist<0):
    dist = 0
  return dist
  
# def isSolvable(grid):
#   count = 0
#   for i in range(0,2):
#     for j in range(i+1,2):
#       if (grid[j, i] > 0 and grid[j, i] > grid[i, j]):
#         count = count + 1
#   if(count%2 == 0):
#     return True
#   else:
#     return False

def solvable(grid):
  tiles=[]
  for x in grid:
    for y in x:
      tiles.append(y)
  count = 0
  for i in range(8):
    for j in range(i+1, 9):
        if tiles[j] and tiles[i] and tiles[i] > tiles[j]:
            count += 1
  return count % 2 == 0
  
def linearConflict(grid):
  acum = 0
  for i in grid:
    if(level(i[0])==level(i[1]) and i[1]<i[0]):
      acum = acum + 1
      if(i[1]==0):
        acum = acum - 1
    if(level(i[0])==level(i[2]) and i[2]<i[0]):
      acum = acum + 1  
      if(i[2]==0):
        acum = acum - 1
    if(level(i[1])==level(i[2]) and i[2]<i[1]):
      acum = acum + 1
      if(i[2]==0):
        acum = acum - 1
  return acum

def level(val):
  if val == 0 or val > 6:
    return 2
  elif val>=1 and val <=3:
    return 0
  else:
    return 1

#function to find an npArray element on a list 
def findQueu(element,listF):
  tempList = copy.deepcopy(listF)
  while not tempList.empty():
    obj = tempList.get()
    obj = obj[1]
    if ((element.tiles==obj.tiles).all()):
      return True
  return False

def findList(tiles,listC):
  for element in listC:
    if ((element==tiles).all()):
      return True
  return False

def delList(tiles,listC):
  index = 0
  for element in listC:
    if ((element==tiles).all()):
      return index
  index = index + 1  
  return False

def a_star_search(start, goal, grids):
  frontier = PriorityQueue() #PriorityQueue with the unexplored grids
  close = []
  priority = start.gVal + start.fVal
  frontier.put(priority, start) #First value of the frontier is the first grid
  cameFrom = [] #The index of a position in cameFrom indicates the referenced grid, the value in that index position is the index to the grid which current came from 
  currentCost = [] #Index in currentCost refers to a grid Index, the value in that index of the list is the distance from the starter grid.
  cameFrom.insert(start.index,start.tiles)
  currentCost.insert(start.index,0)
  steps = 1
  pace = 100
  while not frontier.empty() :
    current = frontier.get() #get the grid with the lowest fVal 
    current = current[1]
    close.append(current.tiles)
    priority = current.gVal + current.fVal
    # frontier.put(priority, current)
    if (current.tiles==goal).all() :
      return current
    #create the list of neighbors
    neighborList = neighbors(current.tiles, current.gVal, grids,current)
    #iterate through neighbors to find the best path
    for neighbor in neighborList:
      #If it is already in the closed set, skip it
      if(not findList(neighbor.tiles,close)):
        #if neighbor on frontier
        if(findQueu(neighbor,frontier)):
          
          tempgVal = current.gVal + 1
          if(tempgVal<neighbor.gVal):
            neighbor.setgVal(tempgVal)
            neighbor.setParent(current)
        else:
          priority = neighbor.gVal + neighbor.fVal
          frontier.put(priority, neighbor)
          
    steps = steps + 1
  print('Sin solucion')  
  return []

def path(finalGrid):
  pathFinal = []
  if(finalGrid != []):
    if(finalGrid != None):
      pathFinal.append(finalGrid)
      if(finalGrid.parent != None):
        finalGrid = finalGrid.parent 
  return pathFinal

global index 
index = 0
